<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Problem-Definition" data-toc-modified-id="Problem-Definition-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Problem Definition</a></span></li><li><span><a href="#Solution-Specification" data-toc-modified-id="Solution-Specification-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Solution Specification</a></span><ul class="toc-item"><li><span><a href="#Part-I:-Representation" data-toc-modified-id="Part-I:-Representation-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Part I: Representation</a></span></li><li><span><a href="#Part-II:-The-World-and-its-Obstacles" data-toc-modified-id="Part-II:-The-World-and-its-Obstacles-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Part II: The World and its Obstacles</a></span></li><li><span><a href="#Part-III:-Agents-and-Their-Attributes" data-toc-modified-id="Part-III:-Agents-and-Their-Attributes-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Part III: Agents and Their Attributes</a></span></li></ul></li><li><span><a href="#The-Code" data-toc-modified-id="The-Code-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>The Code</a></span><ul class="toc-item"><li><span><a href="#Demonstrating-Algorithm-Abilities" data-toc-modified-id="Demonstrating-Algorithm-Abilities-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Demonstrating Algorithm Abilities</a></span></li><li><span><a href="#Sample-Problem---Wumpus-Hunters" data-toc-modified-id="Sample-Problem---Wumpus-Hunters-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Sample Problem - Wumpus Hunters</a></span></li></ul></li><li><span><a href="#Analysis-of-Solution" data-toc-modified-id="Analysis-of-Solution-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Analysis of Solution</a></span></li></ul></div>

## Problem Definition
Many basic AI programs rely on simulating and representing an environment and the agents within it. Often, we are more interested in an algorithms ability to solve a given problem or puzzle than we are in the act of generating any particular environment for it. This final project presents a python class that can, with minimal user input, generate a number of different, grid-based AI environments, with or without agents and with an array of different kinds of obstacles or goals, that can be specified or randomly created. As output, this would generate a succinct representation of the world and it's agents' states, that could then be manipulated by a search algorithm.

As an example:
Input: 5x5 grid, 2 roomba agents, goal: wumpus, obstacle types: pits and dirt
Output: a numpy grid with dictionaries for items, representing a game called "Roombae: The Wumpus Hunters"

## Solution Specification

### Part I: Representation
As the first part of our AI World generator, we need to decide on a representation that can hold the map and information about obstacles and agents in the world. Important here is, of course, that it ought to be easy to manipulate and update this map for a dynamically updating world. I have chosen a combination of numpy arrays and dictionaries for my representation for the following reasons:
- A 3d numpy array, where each slice corresponds to a type of obstacle or goal and where it is located in 2d space, can most succinctly represent the state of a grid-based world (while still being human-legible). This makes it easy to search for things (i.e. if an agent is in spot [0, 5] on slice 0, we simply check slice 1 at the same indexes to see if a goal state is reached). It also allows for relatively straight-forward updating of obstacles using preexisting grid-based algorithms. E.g.: Imagine an obstacle called "growing maze", that starts in a particular spot on the grid and grows outward in a particular pattern. Such a maze could be simulated using cellular automata rules on a 2d grid, allowing us to easily update the slice of our 3d numpy array that represents maze obstacles.
- While this is a great representation for obstacles and any items, we also want our world to be able to have agents in it and for those agents to have attributes. We do not want to end up in a scenario where our orc warrior agent can be found on slice 0, and, once she picks up an axe, we have to transfer her to slice 4. And orcs with axes and a goal item are represented by a value in slice 226. Given this constraint, it seems natural to limit agents of one type to a single slice, with dictionaries representing their attributes. This will allow us to visualize and search agents more effectively.

**Problems this representation addresses:**
- If agents are array-encoded with obstacles, the size of the world representation scales exponentially with the number of agents and attributes. Using a separate agent layer with dictionaries means we only scale linearly with the number of agents/obstacle types.
- Using separate layers for each obstacle type allows easy, grid-based updating. We can simply plug-and-play other algorithms (like cellular automata updating) into the layer of the obstacle.
- Checking for overlap between items, agents and obstacles means simply to take an array-column out of the representation cube/cuboid.
- Using numerical encoding for obstacles means they too can have a limited number of attributes (i.e. a 0 on the wall obstacle-layer could mean no wall, a 1 means a small wall and a 2 means a tall wall).

### Part II: The World and its Obstacles
Obstacle layers are represented by x$*$y sized arrays and obstacles can easily be placed in either random or predetermined locations with specified density. Layers can be selected to avoid collisions with existing layers (i.e. for walls) or not (i.e. sunshine would never collide). The obstacle layer generator supports two special type operations, that generate obstacles with predetermined characteristics:
- **unique**: This keyword generates only a single item of the given value and tags the layer as being unique. This could be used to represent special payoffs or items and checking for a victory condition could be as easy as checking if the unique layer is empty (hence the agent retrieved the item).
- **percept_generator**: This layer type is modelled after the pits and Wumpus in the famous Wumpus World, where obstacles are first placed and later equipped with percepts in all adjacent squares. This could signify the stench of a Wumpus or the breeze of a pit.

### Part III: Agents and Their Attributes
Lastly, agents are represented by special layers filled with dictionaries. Each agent is represented by a separate dictionary, that can be initiated to include different kinds of attributes for each agent (i.e. held objects, abilities, health points, etc.). When placing agents in the world, just like obstacles, they too can avoid being placed inside existing agents or obstacles.

## The Code

In [219]:
import numpy as np
import pandas as pd

pd.options.display.max_columns = None
pd.options.display.width = None

from tabulate import tabulate

In [322]:
class AI_world_generator():
    '''
    This class generates an AI world with arbitrary amounts of agents or 
    obstacles. Layers have to be added in order to avoid collisions.
    '''
    def __init__(self, x, y):
        self.world = np.zeros((x, y, 0))
        self.x = x
        self.y = y
        self.layer_num = 0
        self.decoder = {}
        self.collision_mask = np.full((x, y), False)
        
        
    def add_agent_type(self, name, number_of_agents = 1,
                       locations = None, attributes = {},
                       avoid_collision = False, info = None):
        '''
        This function generates a new agent layer, placing x number of agent
        dictionaries in either pre-specified locations or randomly
        determined places.
        
        Inputs:
            name (str) Name of the agent type. Can be a proper noun like
                "Charlie" or a general descriptor such as "Orc". Required.
            number_of_agents (int) Number of agents of the above type to place
                in the newly generated layer. Default: 1.
            locations (lst) List of tuples of locations to place the agents
                in. If the list of locations is empyt or shorter than the
                number of agents to be placed, the missing locations are
                randomly generated. Optional.
            attributes (dict) Dictionary of attributes and values that the
                agent should start out with. Default: Empty dict.
            info (str) Comment string of additional information that is
                stored in the decoder of the layer.
        '''
        # generate a new layer with empty dictionaries
        layer = [[dict() for y in range(self.y)] for x in range(self.x)]
        layer = np.array(layer)
        
        #checking collision mask for space
        if avoid_collision:
            
            total_spots = self.x * self.y
            if np.sum(self.collision_mask) >= (total_spots-number_of_agents):
                print('There is no space to place agents collision-freely.')
                return
                
        
        # generate agents
        a = 0
        while a < number_of_agents:
            try: 
                a_x, a_y = locations[a]
            except (TypeError, IndexError) as error:
#                 print("No agent location found, generating a random spot instead.")
                a_x = np.random.choice(self.x)
                a_y = np.random.choice(self.y)
            
            #avoiding collsions
            if avoid_collision:
                
                #if collision, generate new location instead
                while self.collision_mask[a_x, a_y]:
                    a_x = np.random.choice(self.x)
                    a_y = np.random.choice(self.y)
                else:
                    agent_name = name.lower() + str(a)
                    layer[a_x,a_y][agent_name] = attributes
                    a += 1
            else:
                agent_name = name.lower() + str(a)
                layer[a_x,a_y][agent_name] = attributes
                a += 1
        
        # update collision mask for future layers
        if avoid_collision:
            self.update_collision_mask(layer)
                    
        
        # adding the agents to the world and updating the decoder
        self.world = np.dstack((self.world, layer))
        self.decoder[self.layer_num] = {}
        self.decoder[self.layer_num]["layer_type"] = "agent"
        self.decoder[self.layer_num]["agent_type"] = name.lower()
        self.decoder[self.layer_num]["agent_count"] = number_of_agents
        
        if info:
            self.decoder[self.layer_num]["info"] = info
        
        self.layer_num += 1
#         print(layer)

      
    
    def add_obstacle_layer(self, name, special_type = None,
                           values = [0, 1], weights = [0.5, 0.5],
                           locations = None, value_key = None,
                           avoid_collision = True, info = None):
        '''
        This function generates a new obstacle layer, placing x number of 
        obstacles in either pre-specified locations or randomly determined
        places.
        There are two ways of using this function: 
        Input values & weights -> generates a random set of obstacles in the
            layer with a distribution of values and their weights. NOTE: Add
            a 0 value and corresponding weight to this, if you desire "no
            obstacle" spots on the map.
        Input values & locations -> If len(values) == len(locations), this
            will place the specific obstacle values in the given locations,
            otherwise it will randomly sample from values (with replacement)
            and place the obstacles in the location spots.
        
        Inputs:
            name (str) Name of the obstacle type. Can be a proper noun like
                "Wumpus" or a general descriptor such as "wall". Required.
            special_type (str) This string idicates to call one of the special
                layer type generators. Unique creates just a single item on
                the layer, percept_generator will add additional values
                around the obstacles that could correspond to percepts (i.e. 
                breezes around pits).
                Optional. Choices: ['unique', 'percept_generator']
            values (lst) List of integers, representing the obstacle types.
                Default: [0, 1]
            weights (lst) List of either probabilities of generating each
                obstacle value on the map or odds of appearance (i.e. [10, 1]
                could mean 10/1 ratio of free space to walls). Does not need
                to be normalized. Default: [0.5, 0.5]
            locations (lst) List of tuples of locations to place obstacles
                in. For explanation of how to use, see above. Optional.
            value_key (dict) Dictionary explaining the meaning of values in
                the layer. Optional.
            info (str) Comment string of additional information that is
                stored in the decoder of the layer.
        ''' 
        layer = np.zeros((self.x, self.y))
        
        # generates a unique obstacle instead (overwrites the below)
        if special_type == 'unique':
            
            #checking collision mask for space
            if avoid_collision:
                total_spots = self.x * self.y
                if np.sum(self.collision_mask) >= (total_spots-1):
                    print('There is no space to place the item collision-freely.')
                    return 
                
            layer = self.generate_unique_layer(values, locations, avoid_collision)
            if info:
                info = "This layer contains a unique item. " + info
            else:
                info = "This layer contains a unique item."
                
        else:        
            # generate normal obstacles
            if not locations:
#                 print("No specific locations given. Populating based on values and weights.")
                normalized_weights = list(weights/np.sum(weights))
                placement = np.random.choice(a = values, size = (self.x, self.y),
                                             p = normalized_weights) 
                placement = np.ma.masked_array(placement, mask = self.collision_mask)
                layer = placement.filled(0)
            else:
                if len(values) == len(locations):
#                     print("Locations and exact values given, placing 'values' in locations.")
                    for l, loc in enumerate(locations):
                        if not self.collision_mask[loc]:
                            layer[loc] = values[l]
                        else:
                            print("Invalid locations, item might not have been placed.")
                else:
#                     print("Locations given, placing random 'values' in locations.")
                    for loc in locations:
                        if not self.collision_mask[loc]:
                            layer[loc] = np.random.choice(values)
                        else:
                            print("Invalid locations, item might not have been placed.")
            
        
        # updating before potentially adding collisions, since the percepts
        # do not count as obstacles.
        if avoid_collision:
            self.update_collision_mask(layer)
            
        
        # adds percepts to the layer if special type selected
        if special_type == 'percept_generator':
            layer, percept = self.add_percepts_to_layer(layer)
            if value_key and (percept not in value_key.keys()):
                value_key[percept] = str(name) + '-percepts'
            elif not value_key:
                value_key = {percept: str(name) + '-percepts'}
                
            if info:
                info = "This layer contains additional percepts. " + info
            else:
                info = "This layer contains additional percepts."
                
#         print(layer)

        # adding the agents to the world and updating the decoder
        self.world = np.dstack((self.world, layer))
        self.decoder[self.layer_num] = {}
        self.decoder[self.layer_num]["layer_type"] = "obstacle"
        self.decoder[self.layer_num]["obstacle_type"] = name.lower()
        
        if value_key:
            self.decoder[self.layer_num]["value_key"] = value_key
        if info:
            self.decoder[self.layer_num]["info"] = info
            
        
        self.layer_num += 1
    
    
        
## LOCATOR FUNCTIONS
## --------------------------------
        
    def get_agent_loc(self, layer_id):
        '''
        Function that returns a list of tuples for all agent locations and 
        their names on the selected layer.
        
        Input:
            layer_id (int) Integer that specifies the index of the layer to
                be searched for agents. The layer has to be of type "agent".
                Required.
        '''
        if self.decoder[layer_id]['layer_type'] != 'agent':
            print("This isn't an agent layer!")
            return None
        else:
            x_loc, y_loc = np.where(self.world[:,:,layer_id] != {})
            final_agent_locs = []
            
            for x, y in zip(x_loc, y_loc):
                agents = self.world[x,y,layer_id].copy()
                
                for key, value in agents.items():
                    final_agent_locs.append( (x, y, key) )
                    
            return final_agent_locs
        
        
    def get_obstacle_loc(self, layer_id, verbose = False):
        '''
        Function that returns a list of tuples for all obstacle locations and 
        their values on the selected layer.
        
        Input:
            layer_id (int) Integer that specifies the index of the layer to
                be searched for obstacles. The layer has to be of type
                "obstacle". Required.
            verbose (bool) If True and available, this prints the value_key
                of the layer for help.
        '''
        if self.decoder[layer_id]['layer_type'] != 'obstacle':
            print("This isn't an obstacle layer!")
            return None
        
        if verbose:
            val_key = self.decoder[layer_id].get('value_key', None)
            if val_key:
                print(val_key)
            else:
                print("No value key available for this layer.")
            
        x_loc, y_loc = np.where(self.world[:,:,layer_id] != 0)
        obstacle_locs = []

        for x, y in zip(x_loc, y_loc):
            obs_val = self.world[x,y,layer_id]
            obstacle_locs.append( (x, y, obs_val) )

        return obstacle_locs

    
    
## SPECIAL OBSTACLE LAYER FUNCTIONS
## --------------------------------
        
    def generate_unique_layer(self, values, locations, avoid_collision):
        '''
        Generates a unique item on a layer either based on the first
        provided location or a randomly chosen spot.
        
        Inputs:
            see parent function above
        '''
        layer = np.zeros((self.x, self.y))
        
        
        if locations:
            x, y = locations[0]
        else:
            x = np.random.choice(range(self.x))
            y = np.random.choice(range(self.y))
        
        if avoid_collision:
            while self.collision_mask[x, y]:
                x = np.random.choice(range(self.x))
                y = np.random.choice(range(self.y))
            else:
                layer[x, y] = values[0]
            
        return layer
    
    
    def add_percepts_to_layer(self, layer):
        '''
        Adds percepts to the layer. Each obstacle on the layer gets a
        percept generating value around it (i.e. a pit of value 3, would
        get value 4s in adjacent squares, representing the breeze it
        generates).
        '''
        percept = np.max(layer) + 1
        
        x_loc, y_loc = np.where(layer != 0)
        obstacle_locs = zip(x_loc, y_loc)
        
        #generating the surrounding squares
        for loc in obstacle_locs:
            surroundings = [(loc[0]-1, loc[1]), (loc[0]+1, loc[1]),
                            (loc[0], loc[1]+1), (loc[0], loc[1]-1)]
            
            #checking for out of bounds
            for sur in surroundings:
                if ( (sur[0] < self.x and sur[0] >= 0) and
                     (sur[1] < self.y and sur[1] >= 0) ):
                    
                    #checking for overlap with existing obstacles
                    if layer[sur] == 0:
                        layer[sur] = percept
        
        return layer, percept
    
    
## COLLISION MASK GENERATOR
## ------------------------  
    def update_collision_mask(self, layer):
        '''
        Creates a mask of all existing collisions on the previous layers to
        avoid placing new agents or obstacles on top of existing ones.
        '''
        collision_points = []

        x_points, y_points = np.where( (layer != 0) & (layer != {}) )
        
        for x, y in zip(x_points, y_points):
            collision_points.append( (x, y) )

        for collision_loc in collision_points:
            self.collision_mask[collision_loc] = True

                
## VISUALIZATION FUNCTION FOR DEBUGGING
## ------------------------------------

    def visualize_layers(self):
        '''
        Prints the items in all layer positions, imposed on a 2d surface.
        '''
        two_2_world = np.empty( (self.x, self.y), dtype = '<U50')
        
        for x in range(self.x):
            for y in range(self.y):
                items = ""
                for layer in range(self.layer_num):
                    item = self.world[x, y, layer]
                    if item != 0 and item != {}:
                        if isinstance(item, dict):
                            keys = [k for k, v in item.items()]
                            item = str(keys)[1:-1]
                        else:
                            val_key = self.decoder[layer].get('value_key', None)
                            if val_key and (int(item) in val_key.keys()):
                                item = val_key[int(item)]
                            else:
                                item = int(item)

                        items = items + '\n' + str(item)
                two_2_world[x, y] = items
        
        pdtabulate = lambda df:tabulate(df, tablefmt='grid', stralign = 'center')
        print(pdtabulate(two_2_world))
        

### Demonstrating Algorithm Abilities
Using the above class functions similar to the way Neural nets can be built up in Tensorflow, where each individual layer can be added after generating the initial world. The decoder functions as a way of keeping track of each individual layer's contents.

In [325]:
world = AI_world_generator(5, 6)

world.add_agent_type("orc", 20, [(0,0), (0,0)], avoid_collision = True,
                         info = "Just an ordinary horde of orcs.")
world.add_agent_type("warrior", 1, [(0,0)], attributes = {'sword': 1, 'treasure': None},
                         avoid_collision = True,
                         info = "A brave warrior with a sword and no treasure.")
world.add_obstacle_layer("wall", values = [1, 2, 1], locations = [(0,0), (4,4), (2,1)],
                         value_key = {1: "half-wall", 2: "tall wall"},
                         info = "This layer has 3 walls in it.")
world.add_obstacle_layer("Wumpus", values = [0, 1], weights = [0.9, 0.1],
                         info = "Wumpus layer with weights given as percentages.")
world.add_obstacle_layer("Wumpus", values = [0, 1], weights = [10, 1],
                         info = "Wumpus layer with weights given as ratios.")
world.add_obstacle_layer("gold", values = [1000], special_type = 'unique',
                         avoid_collision = False,
                         info = "MY PRECIOUS.")
world.add_obstacle_layer("pit", values = [1, 2, 1], locations = [(0,0), (4,4), (2,1)],
                         special_type = 'percept_generator',
                         info = "Some breezy pits.")
# world.visualize_layers()
# print(world.collision_mask)
world.decoder

Invalid locations, item might not have been placed.
Invalid locations, item might not have been placed.
Invalid locations, item might not have been placed.
Invalid locations, item might not have been placed.
Invalid locations, item might not have been placed.


{0: {'layer_type': 'agent',
  'agent_type': 'orc',
  'agent_count': 20,
  'info': 'Just an ordinary horde of orcs.'},
 1: {'layer_type': 'agent',
  'agent_type': 'warrior',
  'agent_count': 1,
  'info': 'A brave warrior with a sword and no treasure.'},
 2: {'layer_type': 'obstacle',
  'obstacle_type': 'wall',
  'value_key': {1: 'half-wall', 2: 'tall wall'},
  'info': 'This layer has 3 walls in it.'},
 3: {'layer_type': 'obstacle',
  'obstacle_type': 'wumpus',
  'info': 'Wumpus layer with weights given as percentages.'},
 4: {'layer_type': 'obstacle',
  'obstacle_type': 'wumpus',
  'info': 'Wumpus layer with weights given as ratios.'},
 5: {'layer_type': 'obstacle',
  'obstacle_type': 'gold',
  'info': 'This layer contains a unique item. MY PRECIOUS.'},
 6: {'layer_type': 'obstacle',
  'obstacle_type': 'pit',
  'value_key': {1.0: 'pit-percepts'},
  'info': 'This layer contains additional percepts. Some breezy pits.'}}

### Sample Problem - Wumpus Hunters

In the following I demonstrate a use of the world generator in recreating the famous Wumpus world, with a small twist.

In [363]:
hunter_world = AI_world_generator(5,10)
hunter_world.add_agent_type('Roomba', 1, [(0,0)], {'kills': 0, 'shots': 30},
                            avoid_collision = True,
                            info = 'This is a roomba hunter. It hunts Wumpi for sports.')
hunter_world.add_obstacle_layer('wall', values = [0, 1], weights = [0.8, 0.2],
                                value_key = {1: '====\n===='},
                            info = 'This layer contains walls the Roomba\'s have to get around.')
hunter_world.add_obstacle_layer('pit', special_type = 'percept_generator',
                                values = [0, 1], weights = [0.8, 0.1], 
                                value_key = {1: 'lava', 2: '~ ~'}, 
                                info = 'These are terrible pits of lava that kill the Roombas.')
hunter_world.add_obstacle_layer('Wumpus', values = [0, 1], weights = [1-(24/100), 24/100],
                                value_key = {1: "(v0_0)v\n _| |_ "},
                                info = 'These are the famous Wumpi, to be hunted by Roombas.')

hunter_world.visualize_layers()
hunter_world.decoder

+-----------+---------+------+---------+---------+------+---------+------+---------+---------+
| 'roomba0' | (v0_0)v | ==== |         |         |      | (v0_0)v | ~ ~  | (v0_0)v | (v0_0)v |
|           |  _| |_  | ==== |         |         |      |  _| |_  |      |  _| |_  |  _| |_  |
+-----------+---------+------+---------+---------+------+---------+------+---------+---------+
|           | (v0_0)v |      |         |         | ==== |   ~ ~   | lava |   ~ ~   |         |
|           |  _| |_  |      |         |         | ==== | (v0_0)v |      | (v0_0)v |         |
|           |         |      |         |         |      |  _| |_  |      |  _| |_  |         |
+-----------+---------+------+---------+---------+------+---------+------+---------+---------+
|           |         |      | (v0_0)v |         | ==== |         | ~ ~  |  lava   |   ~ ~   |
|           |         |      |  _| |_  |         | ==== |         |      |         |         |
|           |         |      |         |         |

{0: {'layer_type': 'agent',
  'agent_type': 'roomba',
  'agent_count': 1,
  'info': 'This is a roomba hunter. It hunts Wumpi for sports.'},
 1: {'layer_type': 'obstacle',
  'obstacle_type': 'wall',
  'value_key': {1: '====\n===='},
  'info': "This layer contains walls the Roomba's have to get around."},
 2: {'layer_type': 'obstacle',
  'obstacle_type': 'pit',
  'value_key': {1: 'lava', 2: '~ ~'},
  'info': 'This layer contains additional percepts. These are terrible pits of lava that kill the Roombas.'},
 3: {'layer_type': 'obstacle',
  'obstacle_type': 'wumpus',
  'value_key': {1: '(v0_0)v\n _| |_ '},
  'info': 'These are the famous Wumpi, to be hunted by Roombas.'}}

In [332]:
print(hunter_world.get_agent_loc(0))
print("-----")
print(hunter_world.get_obstacle_loc(1, verbose = True))
print("-----")
print(hunter_world.get_obstacle_loc(2, verbose = True))
print("-----")
print(hunter_world.get_obstacle_loc(3, verbose = True))

[(0, 0, 'roomba0')]
-----
{1: 'wall'}
[(0, 3, 1), (0, 6, 1), (0, 9, 1), (1, 4, 1), (1, 5, 1), (2, 2, 1), (2, 4, 1), (2, 6, 1), (3, 4, 1), (3, 5, 1), (4, 0, 1), (4, 2, 1), (4, 4, 1), (4, 7, 1), (4, 8, 1), (5, 0, 1), (5, 1, 1), (5, 9, 1), (6, 0, 1), (6, 5, 1), (6, 8, 1), (7, 1, 1), (7, 2, 1), (7, 5, 1), (7, 6, 1), (7, 7, 1), (7, 8, 1), (8, 0, 1), (8, 1, 1)]
-----
{1: 'lava', 2: '~~~'}
[(0, 0, 2), (0, 1, 2), (0, 2, 1), (0, 3, 2), (0, 7, 2), (1, 0, 1), (1, 1, 2), (1, 2, 2), (1, 6, 2), (1, 7, 1), (1, 8, 2), (2, 0, 2), (2, 7, 2), (2, 8, 2), (3, 1, 2), (3, 7, 2), (3, 8, 1), (3, 9, 2), (4, 0, 2), (4, 1, 1), (4, 2, 2), (4, 3, 2), (4, 8, 2), (5, 1, 2), (5, 2, 2), (5, 3, 1), (5, 4, 2), (5, 7, 2), (6, 0, 2), (6, 1, 1), (6, 2, 2), (6, 3, 2), (6, 6, 2), (6, 7, 1), (6, 8, 2), (7, 1, 2), (7, 7, 2), (7, 8, 2), (8, 1, 2), (8, 2, 2), (8, 3, 2), (8, 7, 2), (8, 8, 1), (8, 9, 2), (9, 0, 2), (9, 1, 1), (9, 2, 1), (9, 3, 1), (9, 4, 2), (9, 8, 2)]
-----
{1: 'WUMPUS'}
[(1, 6, 1), (1, 8, 1), (3, 7, 1), (4, 6, 1)